In [ ]:
reload_lamb()

# Variable free binding a la Jacobson

### Notebook author: Kyle Rawlins

This notebook implements (a bit of) the semantic side of Jacobson's account of binding and pronouns.  In so doing, we can get a better idea of how it works, as well as what work is carried by the pure type theory, and what work is carried by the syntactic side of things.  (I don't implement the syntax here.)

>Jacobson, Pauline. "[Towards a Variable-Free Semantics](http://link.springer.com/article/10.1023%2FA%3A1005464228727)", Linguistics and Philosophy 22, 1999. pp. 117-184.

The basic idea (in the end) is that rather than representing binding via indices, we can use type-shifts to hold out an argument slot where a pronoun appeared for later saturation.  I follow the development of the proposal in the article, by first using function composition.

(Warning: Because of the lack of syntax, this notebook is far from being a complete version of Jacobson's account!)

**Step 1**: define a function composition operation on meta-language functions (LFun objects)

* There are multiple ways of doing this.  I have chosen to construct a combinator of the correct type.  The function `geach_combinator` does this (see below for the output)
* This is effectively an implementation of the geach rule.

In [ ]:
geach = %te L g_<Y,Z> : L f_<X,Y> : L x_X : g(f(x))

# use the Geach rule to implement function composition as a python function.
def fc_meta(g, f):
    return geach(g)(f).reduce_all()

Just as an example,here's what this combinator looks like for `X=e`, `Y=e`, `Z=t`, a case that will be common in the variable-free analysis of binding.

In [ ]:
geach.try_adjust_type(tp("<<e,t>,<<e,e>,?>>"))

In [ ]:
%%lamb 
f1 = L x_e : Cat(x)
f2 = L x_e : x_e

In [ ]:
geach(f1).reduce_all()

In [ ]:
fc_meta(f1,f2).derivation

In [ ]:
# this trick is called "monkey patching".  It lets us use * for function composition in the metalanguage.
# note that it isn't really necessary here as function composition is now implemented as a part of the built in LFun class

# meta.LFun.__mul__ = fc_meta

In [ ]:
f1 * f2

In [ ]:
result = None
try:
    result = f2 * f1 # should generate a type mismatch
except types.TypeMismatch as e:
    result = e
result

Next, we can use the Geach combinator to add a function composition operation to the composition system.  Internally, this does something like the above `fc_meta` function, but it is more straightforward to let the wrapper function `add_binary_rule` construct a composition rule from the combinator.

In [ ]:
# add the FC rule to the composition system.  Note that we don't even need to use `fc_meta` here; can just pass the geach combinator in
system = lang.td_system.copy()
system.add_binary_rule(geach, "FC")
lang.set_system(system)
system

**Step 2**: treat pronouns as identity functions, and allow type lifting for names.

(I have simply given the regular and type-lifted version of `Mary` under different names below.)

In [ ]:
%%lamb
||him|| = L x_e  : x
||mary1|| = Mary_e
||mary2|| = L f_<e,t>: f(Mary_e)
||saw|| = L x_e : L y_e : Saw(y,x)

In [ ]:
saw * him

In [ ]:
mary2 * (saw * him)

In [ ]:
(mary2 * (saw * him)).tree()

This seems exactly right, with the abstraction over `x` corresponding to the intuition that `him` is free in this clause.

However, Jacobson points out the following problem with this account: what if the type of the name is not lifted?

In [ ]:
mary1 * (saw * him)

**Step 3**: rather than using a binary operation of function composition, use a unary type-shift that directly involves the geach combinator.

For a really general treatment of this we'd need a bunch more syntactic work (that Jacobson does), and a family of geach combinators.  I will just assume two geach combinators, and one lifting operation.

In [ ]:
lift_combinator_t = %te L f_X : L g_<X,t> : g(f)
lift_combinator_t

In [ ]:
lift_combinator_t(mary1.content).reduce()

In [ ]:
g_e_combinator = geach.let_type(tp("<?,<<e,?>,?>>"))
g_e_combinator.type

In [ ]:
g_et_combinator = geach.let_type(tp("<?,<<<e,t>,?>,?>>"))
g_et_combinator #.get_type_env() #.type_var_set

In [ ]:
system = lang.td_system.copy()
system.add_unary_rule(lift_combinator_t, "lift_t-shift")
system.add_unary_rule(g_e_combinator, "g_e-shift")
system.add_unary_rule(g_et_combinator, "g_et-shift")
lang.set_system(system)
system

The brute-force way to trigger a unary type-shift is to multiply it by None (`x * None`).  The following examples use that idiom to show that, with the g-shift and type-lifted DP, there is no way to get the wrong result.

Though in principle the non-lifted subject can compose with the g-shifted verb, this is ruled out syntactically (not implemented here); abstracting function composition to the g-rule enables this.

In [ ]:
(((mary2 * None) * ((saw * None) * him))).tree()

To handle binding of pronouns, we add a new typeshift, the z-rule.  Again I will implement this using a combinator.

In [ ]:
z_combinator = %te (λ f_<X,<e,Z>>: (λ g_<e,X>: (λ x_e: f(g(x))(x))))
z_combinator

In [ ]:
z_combinator.try_adjust_type(tp("<<e,<e,t>>,?>"))

In [ ]:
z_combinator(saw.content).reduce_all()

In [ ]:
system = lang.td_system.copy()
system.add_unary_rule(lift_combinator_t, "lift_t-shift")
system.add_typeshift(g_e_combinator, "g_e-shift")
system.add_typeshift(g_et_combinator, "g_et-shift")
system.add_typeshift(z_combinator, "z-shift")
system.typeshift=True
lang.set_system(system)
system


In [ ]:
r = mary2 * (saw * him)
r

Note that `r[1]` is effectively a reflexive reading, which is a condition B violation for the pronoun *him*.  That is, this reading is generated by having the subject bind the direct object (via the z-rule).  Nothing in the system at the moment rules this out, and so these typeshifts alone will overgenerate reflexive readings.

`r[0]` is of course the reading we hoped to generate, where `him` is free.

In [ ]:
r[0].tree()

In [ ]:
r[1].tree()

In [ ]:
%%lamb
||every|| = L f_<e,t> : L g_<e,t> : Forall x_e : f(x) >> g(x)
||man|| = L x_e : Man(x)

In [ ]:
dp = every * man
dp

In [ ]:
(dp * (saw * him))

Once again, we overgenerate a condition B violation reading.  However, this process generates exactly the right readings for bound pronouns not in the local domain of the binder, as Jacobson shows:

In [ ]:
%%lamb
||poss|| = L f_<e,t> : Iota x_e : f(x) # just definite article
||mother|| = L x_e : L y_e : Mother(y,x)

In [ ]:
result = (dp * (saw * (poss * (mother * him))))
result

This account generates 3 readings, 2 of them redundant (but generated via different sequences of typeshifts).  (*note, numbers may not be stable across ILNB version changes*) 

 * `result[1]` and `result[3]` are the (same) bound variable reading that is exactly what we are looking for.
 * `result[0]` and `result[2]` are the (same) free reading that we also want.
 * `result[4]` is an overgenerated, and pathological, reflexive reading where `x1` is `x1`'s own mother.  (*TODO: treatment of possessive is slightly different than Jacobson's...*)

In [ ]:
result[1].tree()